In [14]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse, parse_qs
import time
import re
import os
from tqdm import tqdm
import time
import ast
import re
import hashlib

In [15]:
def safe_filename(title, max_len=120):
    title = re.sub(r'[<>:"/\\|?*]', '', title)

    title = re.sub(r'\s+', ' ', title).strip()

    if len(title) > max_len:
        hash_suffix = hashlib.md5(title.encode()).hexdigest()[:8]
        title = title[:max_len] + "_" + hash_suffix

    return title

In [2]:
session = requests.Session()
session.headers.update({
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0.0.0 Safari/537.36"
    ),
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.9",
    "Connection": "keep-alive",
})

In [ ]:
i = -1
texts_hash = {}
def get_pdf_link(url, output_dir='files'):
    os.makedirs(output_dir, exist_ok=True)
    try:
        global i
        i += 1

        response = session.get(url, timeout=15)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, "html.parser")

        title = soup.select_one(
            "section.department-slider.news_main.news-detail-slider h1"
        )
        title = title.get_text(strip=True) if title else f"sebi_circular_{i}"
        title = re.sub(r'[\\/*?:"<>|]', '', title)
        title = re.sub(r'\s+', ' ', title).strip()
        title = safe_filename(title)

    
        iframe = soup.find("iframe", src=True)

        if iframe is not None:
            src = iframe["src"]
            full_src = urljoin(url, src)

            parsed = urlparse(full_src)
            query_params = parse_qs(parsed.query)

            pdf_url = query_params.get("file", [None])[0]

            # print(pdf_url)
            return {
                "type": "pdf",
                "title": title,
                "pdf_url": pdf_url
            }

        else:
            soup = BeautifulSoup(response.text, "lxml")

            content_div = soup.find("div", class_="table-scrollable")

            if content_div is None:
                print("No text content found")
                return None

            text = content_div.get_text(separator="\n")
            text = re.sub(r'\n\s*\n+', '\n\n', text)
            text = text.replace('\xa0', ' ').strip()

            base = title
            if base in texts_hash.keys():
                title = base +"_"+ str(texts_hash[base])
                texts_hash[base] += 1
            else:
                texts_hash[base] = 1

            filename = os.path.join(output_dir, f"{title}.txt")
            
            with open(filename, "w", encoding="utf-8") as f:
                f.write(text)

            # print("Text saved:", filename)
            return {
                "type": "text",
                "title": title,
                "file": filename
            }

    except Exception as e:
        print("Error: ", url, e)
        return None


In [4]:
page_links = []

BASE = "https://www.sebi.gov.in"
AJAX_URL = BASE + "/sebiweb/ajax/home/getnewslistinfo.jsp"

HEADERS = {
    "User-Agent": "Mozilla/5.0",
    "Content-Type": "application/x-www-form-urlencoded",
    "Referer": "https://www.sebi.gov.in/sebiweb/home/HomeAction.do"
}

session = requests.Session()

# Initial GET sets cookies
session.get(
    "https://www.sebi.gov.in/sebiweb/home/HomeAction.do",
    params={"doListing": "yes", "sid": "1", "ssid": "7", "smid": "0"},
    headers=HEADERS
)

for page in tqdm(range(0, 111)):
    payload = {
        "nextValue": "1",
        "next": "n",
        "doDirect": str(page),
        "sid": "1",
        "ssid": "7",
        "smid": "0",
        "search": ""
    }

    r = session.post(AJAX_URL, data=payload, headers=HEADERS)
    # print(f"\n--- PAGE {page + 1} ---", r.status_code)

    if r.status_code != 200:
        continue

    soup = BeautifulSoup(r.text, "html.parser")

    links = soup.find_all("a")
    for link in links:
        href = link.get("href")
        if href and href.startswith("https") and "index.html" not in href and  "legal.html" not in href and "https://www.sebi.gov.in/.html" not in href:
            # print(href)
            page_links.append(href)
    time.sleep(0.4)

100%|██████████| 111/111 [01:00<00:00,  1.84it/s]


In [5]:
unique_links = list(set(page_links))
print(len(unique_links), " pages found")

2734  pages found


In [9]:
with open ('list.txt', 'w') as f:
    f.write(str(unique_links))

In [23]:
pdf_links = []
for link in tqdm(unique_links):
    pdf_links.append(get_pdf_link(link))

 10%|█         | 283/2734 [00:30<47:07,  1.15s/it]

No text content found


 27%|██▋       | 741/2734 [01:13<21:29,  1.55it/s]

No text content found


 42%|████▏     | 1153/2734 [02:06<1:01:45,  2.34s/it]

...

 47%|████▋     | 1291/2734 [02:32<58:16,  2.42s/it]  

No text content found


 92%|█████████▏| 2528/2734 [04:24<00:26,  7.70it/s]

No text content found


100%|██████████| 2734/2734 [04:43<00:00,  9.63it/s]


In [ ]:
print(len((pdf_links)))

In [ ]:
final = []
for pdf_link in pdf_links:
    if pdf_link == None:
        continue
    else:
        final.append(pdf_link)

In [ ]:
print(len(final))

In [ ]:
### Fall back in case of kernel failure
with open ('list.txt', 'w') as f:
    f.write(str(final))

In [ ]:
with open('list.txt', 'r') as f:
    final = ast.literal_eval(f.read())

In [ ]:
for pdf_link in final:
    if "pdf_url" in pdf_link and not pdf_link["pdf_url"].startswith("https"):
        pdf_link["pdf_url"] = "https://www.sebi.gov.in/" + pdf_link["pdf_url"]

In [ ]:
for pdf_link in final:
    if "pdf_url" in pdf_link:
        pdf_link["source"] = pdf_link.pop("pdf_url")
    elif "file" in pdf_link:
        pdf_link["source"] = pdf_link.pop("file")

final[2]

In [ ]:
with open("all_pdfs.txt", 'w') as f:
    f.write(str(final))

In [7]:
# READ
with open("all_pdfs.txt", 'r') as f:
    pdf_urls = ast.literal_eval(f.read())

# pdf_urls